<a href="https://colab.research.google.com/github/Open-Reaction-Database/ord-schema/blob/master/examples/3_Liu_Copper_OrgSyn/example_liu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copper-Catalyzed Enantioselective Hydroamination of Alkenes

DOI: 10.15227/orgsyn.095.0080

Org. Synth. 2018, 95, 80-96

**note: there is more detail in the procedure regarding process steps, workup, etc. than can be captured by the current schema, so we fall back on unstructured string fields**


Colab set-up: install schema

In [1]:
try:
    import ord_schema
    import rdkit
except:
    import sys
    !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
    !time conda install -q -y -c rdkit rdkit
    !time conda install -q -y -c anaconda protobuf
    !git clone https://github.com/Open-Reaction-Database/ord-schema.git
    %cd ord-schema
    !python setup.py install
    sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2020-06-09 22:49:33--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-06-09 22:49:33--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  81.12M   181MB/s    in 0.4s    

2020-06-09 22:49:34 (181 MB/s) - ‘Miniconda3-latest-Linux

Import schema and helper functions

In [0]:
from datetime import datetime
from ord_schema.proto import reaction_pb2
from ord_schema.units import UnitResolver
from ord_schema import validations
from ord_schema import message_helpers

unit_resolver = UnitResolver()

# Procedure 1

Procedure 1 to make N,N-Dibenzyl-O-pivaloylhydroxylamine

In [3]:
reaction = reaction_pb2.Reaction()

# 500-mL single-necked flask
reaction.setup.vessel.CopyFrom(
    reaction_pb2.Vessel(
        type=dict(type='ROUND_BOTTOM_FLASK', details='single-necked'), 
        material=dict(type='GLASS'),
        volume=unit_resolver.resolve('500 mL'),
    )
)
reaction.setup.vessel.preparations.add(type='OVEN_DRIED')
reaction.setup.vessel.preparations.add(type='PURGED', details='with nitrogen')
reaction.setup.vessel.attachments.add(type='SEPTUM', details='rubber')
reaction.setup.is_automated = reaction_pb2.Boolean.FALSE
reaction.setup.environment.type = reaction_pb2.ReactionSetup.ReactionEnvironment.FUME_HOOD

# Three components charged to flask initially, in order
reaction.inputs['N,N-dibenzylhydroxylamine'].addition_order = 1
reaction.inputs['N,N-dibenzylhydroxylamine'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='C1=CC=C(C=C1)CN(CC2=CC=CC=C2)O', 
        name='N,N-dibenzylhydroxylamine', 
        prep='custom', 
        prep_details='a few colored or darker crystals, which were present in trace'
            ' amounts, were discarded using standard tweezers',
        vendor='TCI America',
        role='reactant', 
        amount='21.3 g',
        is_limiting=True,
    )
)
reaction.inputs['4-dimethyl-aminopyridine'].addition_order = 2
reaction.inputs['4-dimethyl-aminopyridine'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='n1ccc(N(C)C)cc1', 
        name='4-dimethyl-aminopyridine', 
        prep='none', 
        prep_details='used as received',
        vendor='Sigma Aldrich',
        role='reactant', 
        amount='12.8 g'
    )
)
reaction.inputs['dichloromethane'].addition_order = 3
reaction.inputs['dichloromethane'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='C(Cl)Cl', 
        name='dichloromethane', 
        prep='dried', 
        prep_details='purified by passage under argon pressure through two packed '
        'columns of neutral alumina and copper(II) oxide',
        vendor='J. T. Baker',
        role='solvent', 
        amount='250 ml'
    )
)

# Nothing in the schema can capture the 5 minutes stirring at RT? Then 20 min
# of cooling! Need to use catch-all
reaction.conditions.conditions_are_dynamic = reaction_pb2.Boolean.TRUE
reaction.conditions.details = '''reaction started in ice bath before addition of
pivaloyl chloride, then allowed to warm to 23 C, then stirred for '
an additional 6 hours at RT (23 degC)'''

reaction.inputs['pivaloyl chloride'].addition_order = 4
reaction.inputs['pivaloyl chloride'].addition_speed.type = \
    reaction_pb2.ReactionInput.AdditionSpeed.DROPWISE 
reaction.inputs['pivaloyl chloride'].addition_duration.CopyFrom(
    unit_resolver.resolve('5 min')
)
reaction.inputs['pivaloyl chloride'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CC(C)(C)C(=O)Cl', 
        name='pivaloyl chloride', 
        prep='none', 
        prep_details='used as received',
        vendor='Alfa Aesar',
        role='reactant', 
        amount='12.9 ml'
    )
)
reaction.inputs['pivaloyl chloride'].addition_device.type = reaction_pb2.ReactionInput.AdditionDevice.SYRINGE
reaction.inputs['pivaloyl chloride'].addition_device.details = 'plastic 30-mL syringe'

# System described as a suspension
reaction.notes.is_heterogeneous = reaction_pb2.Boolean.TRUE

# Connected to nitrogen manifold; flask sealed with rubber septum
p_conds = reaction.conditions.pressure
p_conds.control.type = p_conds.PressureControl.SLIGHT_POSITIVE
p_conds.control.details = 'sealed with rubber septum, pierced with needle connected to manifold'
p_conds.atmosphere.type = p_conds.Atmosphere.NITROGEN 
p_conds.atmosphere.details = 'dry nitrogen'

# 4 cm Teflon-coated magnetic stir bar, speed unknown
s_conds = reaction.conditions.stirring 
s_conds.method.type = s_conds.StirringMethod.STIR_BAR 

# Temperature schedule is rather complicated; this assumes that the reaction
# takes place in the 6 hours at room temperautre after warming up...but we should
# capture that it starts on ice!
t_conds = reaction.conditions.temperature
t_conds.setpoint.CopyFrom(reaction_pb2.Temperature(units='CELSIUS', value=0))
t_conds.control.type = t_conds.TemperatureControl.ICE_BATH
t_conds.control.details = '''reaction started in ice bath before addition of pivaloyl 
chloride, then allowed to warm to 23 C, then stirred for additional 6 hours at 
RT (23 degC)'''

# Workup step 1 - add 50 mL saturated aqueous ammonium chloride
workup = reaction.workup.add()
workup.type = workup.ADDITION
workup.input.components.add().CopyFrom(message_helpers.build_compound(
    smiles='O', name='water', amount='50 ml'
))
workup.input.components.add().CopyFrom(message_helpers.build_compound(
    smiles='[NH4+].[Cl-]',
    name='ammonium chloride',
    amount='19.15 g' # estimated from solubility of 383.0 g/L @ 25 degC
))

# Workup step 2 - use 3 x 50 mL DCM to extract product into organic phase
workup = reaction.workup.add()
workup.type = workup.ADDITION
workup.input.components.add().CopyFrom(message_helpers.build_compound(
    smiles='C(Cl)Cl', name='DCM', amount='50 ml'
))
workup = reaction.workup.add()
workup.type = workup.ADDITION
workup.input.components.add().CopyFrom(message_helpers.build_compound(
    smiles='C(Cl)Cl', name='DCM', amount='50 ml'
))
workup = reaction.workup.add()
workup.type = workup.ADDITION
workup.input.components.add().CopyFrom(message_helpers.build_compound(
    smiles='C(Cl)Cl', name='DCM', amount='50 ml'
))
workup = reaction.workup.add()
workup.type = workup.EXTRACTION
workup.details = '3 x 50 mL DCM extraction in 1 L separatory funnel'
workup.keep_phase = 'organic'

# Workup step 3 - rinse with 200 mL DI water
workup = reaction.workup.add()
workup.type = workup.WASH
workup.input.components.add().CopyFrom(message_helpers.build_compound(
    smiles='O', name='water', amount='200 ml'
))

# Workup step 4 - concentrate using rotovap (30 C, 80 mmHg)
workup = reaction.workup.add()
workup.type = workup.CONCENTRATION 
workup.details = 'rotary evaporator (30 degC, 80 mmHg)'

# Workup step 5 - dissolve heterogeneous mixture in DCM
workup = reaction.workup.add()
workup.type = workup.ADDITION 
workup.input.components.add().CopyFrom(message_helpers.build_compound(
    smiles='C(Cl)Cl', name='DCM', amount='50 ml'
))

# Workup step 6 - pass through alumina column and collect fractions
workup = reaction.workup.add()
workup.type = workup.OTHER_CHROMATOGRAPHY
workup.details = '''Aluminum oxide (neutral, powder, reagent-grade) was 
purchased from J.T. Baker. The crude reaction mixture is suspended in 
dichloromethane (50 mL) and is loaded onto a column, with interior diameter of 
roughly 2 inches, packed with alumina (100 g) and wetted with hexanes. 
dichloromethane is used as the eluent, and fractions are collected in Erlenmeyer
 flasks (50 mL each). The desired product typically elutes in fractions 2 
 through 25. The fractions that contain the desired product are combined'''

# Workup step 7 - remove solvent with rotovap and dry under vacuum
workup = reaction.workup.add()
workup.type = workup.CONCENTRATION 
workup.details = 'rotary evaporator (30 degC, 80 mmHg)'
workup = reaction.workup.add()
workup.type = workup.DRY_IN_VACUUM 

## Product and characterization
outcome = reaction.outcomes.add()
outcome.reaction_time.CopyFrom(unit_resolver.resolve('6 h'))
analysis = reaction_pb2.ReactionAnalysis() # for handy enum reference

outcome.analyses['isolated_weight'].type = analysis.WEIGHT
outcome.analyses['isolated_weight'].details = '27.5-28.0 g recovered after workup'

outcome.analyses['1H NMR'].type = analysis.NMR_1H
outcome.analyses['1H NMR'].details = '400 MHz, CDCl3'
outcome.analyses['1H NMR'].processed_data['peaks'].string_value = r'0.92 (s, 9H), 4.06 (s, 4H), 7.23 - 7.34 (m, 6H), 7.40 (d, J = 7.1 Hz, 4H)'
outcome.analyses['1H NMR'].processed_data['peaks'].description = 'List of peaks'

outcome.analyses['13C NMR'].type = analysis.NMR_13C
outcome.analyses['13C NMR'].details = '101 MHz, CDCl3'
outcome.analyses['13C NMR'].processed_data['peaks'].string_value = r'27.1, 38.4, 62.4, 127.7, 128.3, 129.6, 136.2, 176.3'
outcome.analyses['13C NMR'].processed_data['peaks'].description = 'List of peaks'

outcome.analyses['thin film IR'].type = analysis.IR
outcome.analyses['thin film IR'].details = 'neat film, NaCl'
outcome.analyses['thin film IR'].processed_data['peaks'].string_value = r'3064, 3031, 2973, 2932, 2906, 2872, 1751, 1496, 1479, 1456, 1273, 1116, 1029, 738, 698'
outcome.analyses['thin film IR'].processed_data['peaks'].description = 'List of peaks [cm-1]'

outcome.analyses['HRMS'].type = analysis.HRMS
outcome.analyses['HRMS'].details = 'ESI-TOF'
outcome.analyses['HRMS'].processed_data['expected'].float_value = 298.1802
outcome.analyses['HRMS'].processed_data['expected'].description = 'Expected m/z'
outcome.analyses['HRMS'].processed_data['found'].float_value = 298.1794
outcome.analyses['HRMS'].processed_data['found'].description = 'Observed m/z'

outcome.analyses['quantitative NMR'].type = analysis.NMR_1H
outcome.analyses['quantitative NMR'].details = 'Quantitative NMR using 1,1,2,2-tetrachloroethane (>98%, purchased from Alfa Aesar) in CDCl3'

# A single product was desired and characterized
product = outcome.products.add()
product.compound.CopyFrom(
    message_helpers.build_compound(
        smiles='O=C(C(C)(C)C)ON(CC1=CC=CC=C1)CC2=CC=CC=C2', 
        name='N,N-Dibenzyl-O-pivaloylhydroxylamine', 
        amount='27.75 g'
    )
)
product.purity.value = 99
product.compound_yield.CopyFrom(
    reaction_pb2.Percentage(value=93.5, precision=0.5)
)

# Define which analyses were used for which aspects of characterization
product.analysis_identity.extend(['1H NMR', '13C NMR', 'HRMS'])
product.analysis_yield.append('isolated_weight')
product.analysis_purity.append('quantitative NMR')

product.isolated_color = 'white'
product.texture.type = product.Texture.POWDER

reaction.provenance.city = r'Cambridge, MA'
reaction.provenance.doi = r'10.15227/orgsyn.095.0080'
reaction.provenance.publication_url = r'http://www.orgsyn.org/demo.aspx?prep=v95p0080'
reaction.provenance.record_created.time.value = \
    datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
    name='Connor W. Coley', organization='MIT', orcid='0000-0002-8271-8723'
))

errors = validations.validate_message(reaction)
print(errors)
print(reaction)

[]
inputs {
  key: "4-dimethyl-aminopyridine"
  value {
    components {
      identifiers {
        type: SMILES
        value: "n1ccc(N(C)C)cc1"
      }
      identifiers {
        type: NAME
        value: "4-dimethyl-aminopyridine"
      }
      mass {
        value: 12.800000190734863
        units: GRAM
      }
      reaction_role: REACTANT
      preparations {
        type: NONE
        details: "used as received"
      }
      vendor_source: "Sigma Aldrich"
    }
    addition_order: 2
  }
}
inputs {
  key: "N,N-dibenzylhydroxylamine"
  value {
    components {
      identifiers {
        type: SMILES
        value: "C1=CC=C(C=C1)CN(CC2=CC=CC=C2)O"
      }
      identifiers {
        type: NAME
        value: "N,N-dibenzylhydroxylamine"
      }
      mass {
        value: 21.299999237060547
        units: GRAM
      }
      reaction_role: REACTANT
      is_limiting: TRUE
      preparations {
        type: CUSTOM
        details: "a few colored or darker crystals, which were pres

/content/ord-schema/ord_schema/validations.py:162: ValidationWarning: Temperature setpoints should be specified; even if using ambient conditions, estimate room temperature and the precision of your estimate.
  warnings.warn(warning.message)


# Procedure 2

Procedure 2 to make (R)-N,N-Dibenzyl-1-phenylpropan-1-amine

In [4]:
reaction = reaction_pb2.Reaction()

reaction.setup.vessel.CopyFrom(
    reaction_pb2.Vessel(
        type=dict(type='ROUND_BOTTOM_FLASK', details='two-necked'),
        material=dict(type='GLASS'),
        volume=unit_resolver.resolve('250 mL'),
    )
)
reaction.setup.vessel.preparations.add(type='OVEN_DRIED')
reaction.setup.vessel.preparations.add(type='PURGED', details='with nitrogen')
reaction.setup.vessel.attachments.add(type='SEPTUM', details='rubber')
reaction.setup.vessel.attachments.add(type='SEPTUM', details='rubber')
reaction.setup.is_automated = reaction_pb2.Boolean.FALSE
reaction.setup.environment.type = reaction_pb2.ReactionSetup.ReactionEnvironment.FUME_HOOD

# Color change noted during the 10 minute addition of dimethoxy(methyl)silane
reaction.observations.add().CopyFrom(reaction_pb2.ReactionObservation(
     time=unit_resolver.resolve('0 min'), comment='blue'
))
reaction.observations.add().CopyFrom(reaction_pb2.ReactionObservation(
     time=unit_resolver.resolve('10 min'), comment='orange'
))

# Components charged to flask initially, in order
reaction.inputs['N,N-Dibenzyl-O-pivaloylhydroxylamine'].addition_order = 1
reaction.inputs['N,N-Dibenzyl-O-pivaloylhydroxylamine'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='O=C(C(C)(C)C)ON(CC1=CC=CC=C1)CC2=CC=CC=C2', 
        name='N,N-Dibenzyl-O-pivaloylhydroxylamine', 
        prep='synthesized', 
        role='reactant', 
        amount='7.55 g',
    )
)

reaction.inputs['copper(II) acetate'].addition_order = 2
reaction.inputs['copper(II) acetate'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CC(=O)[O-].CC(=O)[O-].[Cu+2]', 
        name='copper(II) acetate', 
        prep='none', 
        prep_details='used as received (97% purity)',
        vendor='Strem',
        role='reagent', 
        amount='38 mg'
    )
)

reaction.inputs['(S)-DTBM-SEGPHOS'].addition_order = 3
reaction.inputs['(S)-DTBM-SEGPHOS'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CC(C)(C)C1=CC(=CC(=C1OC)C(C)(C)C)P(C2=C(C3=C(C=C2)OCO3)C4=C(C=CC5=C4OCO5)P(C6=CC(=C(C(=C6)C(C)(C)C)OC)C(C)(C)C)C7=CC(=C(C(=C7)C(C)(C)C)OC)C(C)(C)C)C8=CC(=C(C(=C8)C(C)(C)C)OC)C(C)(C)C', 
        name='(S)-DTBM-SEGPHOS', 
        prep='none', 
        prep_details='used as received (>94% purity)',
        vendor='Takasago',
        role='reagent', 
        amount='274 mg'
    )
)

reaction.inputs['triphenylphosphine'].addition_order = 4
reaction.inputs['triphenylphosphine'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='c3c(P(c1ccccc1)c2ccccc2)cccc3', 
        name='triphenylphosphine', 
        prep='none', 
        prep_details='used as received (99% purity)',
        vendor='Sigma-Aldrich',
        role='reagent', 
        amount='61 mg'
    )
)

reaction.inputs['trans-β-methylstyrene'].addition_order = 5
reaction.inputs['trans-β-methylstyrene'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='C/C=C/C1=CC=CC=C1', 
        name='trans-β-methylstyrene', 
        prep='none', 
        prep_details='used as received (97% purity, stabilized)',
        vendor='Combi-Blocks or Acros',
        role='reactant', 
        amount='2.50 g',
        is_limiting=True
    )
)

reaction.inputs['THF'].addition_order = 6
reaction.inputs['THF'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='C1CCOC1', 
        name='THF', 
        prep='dried', 
        prep_details='purified by passage under argon pressure through two '
            'packed columns of neutral alumina and copper(II) oxide',
        vendor='J. T. Baker',
        role='solvent', 
        amount='21 ml'
    )
)
reaction.inputs['THF'].addition_device.type = reaction_pb2.ReactionInput.AdditionDevice.SYRINGE

reaction.inputs['dimethoxy(methyl)silane'].addition_order = 5
reaction.inputs['dimethoxy(methyl)silane'].addition_speed.type = reaction_pb2.ReactionInput.AdditionSpeed.DROPWISE
reaction.inputs['dimethoxy(methyl)silane'].addition_duration.CopyFrom(
    unit_resolver.resolve('10 min')
)
reaction.inputs['dimethoxy(methyl)silane'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CO[Si](C)OC', 
        name='dimethoxy(methyl)silane', 
        prep='none', 
        prep_details='used as received (>97% purity, stored at -20 C)',
        vendor='TCI America',
        role='reagent', 
        amount='4.49 g'
    )
)

# Nothing in the schema can capture the dynamic temperature
reaction.conditions.conditions_are_dynamic = reaction_pb2.Boolean.TRUE
reaction.conditions.details = '''After dropwise addition of dimethoxy(methyl)-
silane into reaction mixture in water bath, the vessel was removed and allowed
to stir for an additional 12 hours'''

# Connected to nitrogen manifold; flask sealed with rubber septum
p_conds = reaction.conditions.pressure
p_conds.control.type = p_conds.PressureControl.SLIGHT_POSITIVE
p_conds.control.details = 'sealed with rubber septum, pierced with needle connected to manifold'
p_conds.atmosphere.type = p_conds.Atmosphere.NITROGEN 
p_conds.atmosphere.details = 'dry nitrogen'

# 1 cm Teflon-coated magnetic stir bar, speed unknown
s_conds = reaction.conditions.stirring 
s_conds.method.type = s_conds.StirringMethod.STIR_BAR 

# Temperature is RT the whole time, but uses a water bath for first 10 min
t_conds = reaction.conditions.temperature
t_conds.setpoint.CopyFrom(reaction_pb2.Temperature(units='CELSIUS', value=23))
t_conds.control.type = t_conds.TemperatureControl.WATER_BATH
t_conds.control.details = 'removed from water bath after 10 min for 12 hour duration'

# TODO(ccoley) Workup steps are currently missing!

# One outcome at 12 hours characterized
outcome = reaction.outcomes.add()
outcome.reaction_time.CopyFrom(unit_resolver.resolve('12 h'))
analysis = reaction_pb2.ReactionAnalysis() # for handy enum reference

outcome.analyses['isolated_weight'].type = analysis.WEIGHT
outcome.analyses['isolated_weight'].details = '5.72 g recovered after workup'

outcome.analyses['1H NMR'].type = analysis.NMR_1H
outcome.analyses['1H NMR'].details = '400 MHz, CDCl3'
outcome.analyses['1H NMR'].processed_data['peaks'].string_value = r'1.01 (t, J = 7.3 Hz, 3H), 1.89 (ddq, J = 14.2, 7.2, 7.1 Hz, 1H), 2.17 (ddq, J = 14.1, 7.2, 7.1 Hz, 1H), 3.24 (d, J = 13.9 Hz, 2H), 3.68 (t, J = 7.5 Hz, 1H), 3.91 (d, J = 13.8 Hz, 2H), 7.50-7.28 (m, 15H)'

outcome.analyses['13C NMR'].type = analysis.NMR_13C
outcome.analyses['13C NMR'].details = '101 MHz, CDCl3'
outcome.analyses['13C NMR'].processed_data['peaks'].string_value = r'11.9, 24.4, 53.8, 63.8, 126.8, 127.0, 128.0, 128.3, 128.9, 129.1, 139.1, 140.6'

outcome.analyses['thin film IR'].type = analysis.IR
outcome.analyses['thin film IR'].details = 'neat film, NaCl'
outcome.analyses['thin film IR'].processed_data['peaks'].string_value = r'3083, 3061, 3027, 2962, 2932, 2873, 2802, 1948, 1872, 1809, 1602, 1493, 1453, 761, 742 cm-1'

outcome.analyses['HRMS'].type = analysis.HRMS
outcome.analyses['HRMS'].details = 'ESI-TOF'
outcome.analyses['HRMS'].processed_data['expected'].float_value = 316.2060
outcome.analyses['HRMS'].processed_data['found'].float_value = 316.2049

outcome.analyses['quantitative NMR'].type = analysis.NMR_1H
outcome.analyses['quantitative NMR'].details = 'Quantitative NMR using ferrocene (98%, purchased from Sigma-Aldrich, recrystallized from pentane) in CDCl3'

outcome.analyses['chiral HPLC'].type = analysis.LC
outcome.analyses['chiral HPLC'].details = 'HPLC (Daicel Chiralpak OD-H column), eluting with 4% isopropanol in hexanes at 0.6 mL/min: 10.9 min (minor), 13.4 min (major), 98% ee for the first run and 99% ee for the second run'

outcome.analyses['specific rotation'].type = analysis.CD
outcome.analyses['specific rotation'].details = 'Specific rotation: [α]D = +108 (c = 1.0, chloroform)'

product = outcome.products.add()
product.compound.CopyFrom(
    message_helpers.build_compound(
        smiles='CC[C@@H](N(CC1=CC=CC=C1)CC2=CC=CC=C2)C3=CC=CC=C3', 
        name='(R)-N,N-Dibenzyl-1-phenylpropan-1-amine', 
        amount='5.72 g'
    )
)
product.compound_yield.CopyFrom(
    reaction_pb2.Percentage(value=86)
)
product.selectivity.type = reaction_pb2.Selectivity.SelectivityType.EE
product.selectivity.value = 98
product.purity.value = 99

product.analysis_identity.extend(['1H NMR', '13C NMR', 'HRMS'])
product.analysis_yield.append('isolated_weight')
product.analysis_purity.append('quantitative NMR')
product.analysis_selectivity.extend(['chiral HPLC', 'specific rotation'])

product.isolated_color = 'colorless'
product.texture.type = product.Texture.OIL

reaction.provenance.city = r'Cambridge, MA'
reaction.provenance.doi = r'10.15227/orgsyn.095.0080'
reaction.provenance.publication_url = r'http://www.orgsyn.org/demo.aspx?prep=v95p0080'
reaction.provenance.record_created.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
    name='Connor W. Coley', organization='MIT', orcid='0000-0002-8271-8723'
))

errors = validations.validate_message(reaction)
print(errors)
print(reaction)


[]
inputs {
  key: "(S)-DTBM-SEGPHOS"
  value {
    components {
      identifiers {
        type: SMILES
        value: "CC(C)(C)C1=CC(=CC(=C1OC)C(C)(C)C)P(C2=C(C3=C(C=C2)OCO3)C4=C(C=CC5=C4OCO5)P(C6=CC(=C(C(=C6)C(C)(C)C)OC)C(C)(C)C)C7=CC(=C(C(=C7)C(C)(C)C)OC)C(C)(C)C)C8=CC(=C(C(=C8)C(C)(C)C)OC)C(C)(C)C"
      }
      identifiers {
        type: NAME
        value: "(S)-DTBM-SEGPHOS"
      }
      mass {
        value: 274.0
        units: MILLIGRAM
      }
      reaction_role: REAGENT
      preparations {
        type: NONE
        details: "used as received (>94% purity)"
      }
      vendor_source: "Takasago"
    }
    addition_order: 3
  }
}
inputs {
  key: "N,N-Dibenzyl-O-pivaloylhydroxylamine"
  value {
    components {
      identifiers {
        type: SMILES
        value: "O=C(C(C)(C)C)ON(CC1=CC=CC=C1)CC2=CC=CC=C2"
      }
      identifiers {
        type: NAME
        value: "N,N-Dibenzyl-O-pivaloylhydroxylamine"
      }
      mass {
        value: 7.550000190734863
        

# Procedure 3

Procedure 3 to make  (R)-N,N-Dibenzyl-2,3,3-trimethylbutan-1-amine

In [5]:
reaction = reaction_pb2.Reaction()

reaction.setup.vessel.CopyFrom(
    reaction_pb2.Vessel(
        type=dict(type='ROUND_BOTTOM_FLASK', details='two-necked'),
        material=dict(type='GLASS'),
        volume=unit_resolver.resolve('250 mL'),
    )
)
reaction.setup.vessel.preparations.add(type='OVEN_DRIED')
reaction.setup.vessel.preparations.add(type='PURGED', details='with nitrogen')
reaction.setup.vessel.attachments.add(type='SEPTUM', details='rubber')
reaction.setup.vessel.attachments.add(type='SEPTUM', details='rubber')
reaction.setup.is_automated = reaction_pb2.Boolean.FALSE
reaction.setup.environment.type = reaction_pb2.ReactionSetup.ReactionEnvironment.FUME_HOOD

# Color change noted during the 10 minute addition of dimethoxy(methyl)silane
reaction.observations.add().CopyFrom(reaction_pb2.ReactionObservation(
    time=unit_resolver.resolve('0 min'), comment='blue'
))
reaction.observations.add().CopyFrom(reaction_pb2.ReactionObservation(
    time=unit_resolver.resolve('10 min'), comment='orange'
))

# Components charged to flask initially, in order
reaction.inputs['N,N-Dibenzyl-O-pivaloylhydroxylamine'].addition_order = 1
reaction.inputs['N,N-Dibenzyl-O-pivaloylhydroxylamine'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='O=C(C(C)(C)C)ON(CC1=CC=CC=C1)CC2=CC=CC=C2', 
        name='N,N-Dibenzyl-O-pivaloylhydroxylamine', 
        prep='synthesized', 
        role='reactant', 
        amount='9.09 g',
    )
)

reaction.inputs['copper(II) acetate'].addition_order = 2
reaction.inputs['copper(II) acetate'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CC(=O)[O-].CC(=O)[O-].[Cu+2]', 
        name='copper(II) acetate', 
        prep='none', 
        prep_details='used as received (97% purity)',
        vendor='Strem',
        role='reagent', 
        amount='46 mg'
    )
)

reaction.inputs['(S)-DTBM-SEGPHOS'].addition_order = 3
reaction.inputs['(S)-DTBM-SEGPHOS'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CC(C)(C)C1=CC(=CC(=C1OC)C(C)(C)C)P(C2=C(C3=C(C=C2)OCO3)C4=C(C=CC5=C4OCO5)P(C6=CC(=C(C(=C6)C(C)(C)C)OC)C(C)(C)C)C7=CC(=C(C(=C7)C(C)(C)C)OC)C(C)(C)C)C8=CC(=C(C(=C8)C(C)(C)C)OC)C(C)(C)C', 
        name='(S)-DTBM-SEGPHOS', 
        prep='none', 
        prep_details='used as received (>94% purity)',
        vendor='Takasago',
        role='reagent', 
        amount='330 mg'
    )
)

reaction.inputs['triphenylphosphine'].addition_order = 4
reaction.inputs['triphenylphosphine'].components.add().CopyFrom(
  message_helpers.build_compound(
    smiles='c3c(P(c1ccccc1)c2ccccc2)cccc3', 
    name='triphenylphosphine', 
    prep='none', 
    prep_details='used as received (99% purity)',
    vendor='Sigma-Aldrich',
    role='reagent', 
    amount='74 mg'
  )
)

reaction.inputs['2,3,3-trimethyl-1-butene'].addition_order = 5
reaction.inputs['2,3,3-trimethyl-1-butene'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CC(=C)C(C)(C)C', 
        name='2,3,3-trimethyl-1-butene', 
        prep='none', 
        prep_details='used as received (98% purity)',
        vendor='Sigma-Aldrich',
        role='reactant', 
        amount='2.50 g',
        is_limiting=True
    )
)

reaction.inputs['THF'].addition_order = 6
reaction.inputs['THF'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='C1CCOC1', 
        name='THF', 
        prep='dried', 
        prep_details='purified by passage under argon pressure through two packed columns of neutral alumina and copper(II) oxide',
        vendor='J. T. Baker',
        role='solvent', 
        amount='25 ml'
    )
)
reaction.inputs['THF'].addition_device.type = reaction_pb2.ReactionInput.AdditionDevice.SYRINGE

reaction.inputs['dimethoxy(methyl)silane'].addition_order = 5
reaction.inputs['dimethoxy(methyl)silane'].addition_speed.type = reaction_pb2.ReactionInput.AdditionSpeed.DROPWISE
reaction.inputs['dimethoxy(methyl)silane'].addition_duration.CopyFrom(
  unit_resolver.resolve('10 min')
)
reaction.inputs['dimethoxy(methyl)silane'].components.add().CopyFrom(
    message_helpers.build_compound(
        smiles='CO[Si](C)OC', 
        name='dimethoxy(methyl)silane', 
        prep='none', 
        prep_details='used as received (>97% purity, stored at -20 C)',
        vendor='TCI America',
        role='reagent', 
        amount='5.41 g'
    )
)
reaction.inputs['dimethoxy(methyl)silane'].addition_device.type = reaction_pb2.ReactionInput.AdditionDevice.SYRINGE

# Nothing in the schema can capture the dynamic temperature
reaction.conditions.conditions_are_dynamic = reaction_pb2.Boolean.TRUE
reaction.conditions.details = '''After dropwise addition of dimethoxy(methyl)-
silane into reaction mixture in water bath, the vessel was removed and allowed
to stir for an additional 12 hours'''

# Connected to nitrogen manifold; flask sealed with rubber septum
p_conds = reaction.conditions.pressure
p_conds.control.type = p_conds.PressureControl.SLIGHT_POSITIVE
p_conds.control.details = 'sealed with rubber septum, pierced with needle connected to manifold'
p_conds.atmosphere.type = p_conds.Atmosphere.NITROGEN 
p_conds.atmosphere.details = 'dry nitrogen'

# 1 cm Teflon-coated magnetic stir bar, speed unknown
s_conds = reaction.conditions.stirring 
s_conds.method.type = s_conds.StirringMethod.STIR_BAR 

# Temperature held at 40 C using oil bath
t_conds = reaction.conditions.temperature
t_conds.setpoint.CopyFrom(reaction_pb2.Temperature(units='CELSIUS', value=40))
t_conds.control.type = t_conds.TemperatureControl.OIL_BATH

# TODO(ccoley) Workup and quench is not recorded here yet

outcome = reaction.outcomes.add()
outcome.reaction_time.CopyFrom(unit_resolver.resolve('12 h'))
analysis = reaction_pb2.ReactionAnalysis() # for handy enum reference

outcome.analyses['isolated_weight'].type = analysis.WEIGHT
outcome.analyses['isolated_weight'].details = '6.54 g recovered after workup'

outcome.analyses['1H NMR'].type = analysis.NMR_1H
outcome.analyses['1H NMR'].details = '400 MHz, CDCl3'
outcome.analyses['1H NMR'].processed_data['peaks'].string_value = r'0.81 (s, 9H), 0.90 (d, J = 6.7 Hz, 3H), 1.49 (dq, J = 7.1, 3.4 Hz, 1H), 2.13 (dd, J = 12.3, 10.5 Hz, 1H), 2.39 (dd, J = 12.2, 2.8 Hz, 1H), 3.21 (d, J = 13.7 Hz, 2H), 3.83 (d, J = 13.7 Hz, 2H), 7.23 (t, J = 7.3 Hz, 2H), 7.31 (t, J = 7.4 Hz, 4H), 7.37 (d, J = 7.5 Hz, 4H)'

outcome.analyses['13C NMR'].type = analysis.NMR_13C
outcome.analyses['13C NMR'].details = '101 MHz, CDCl3'
outcome.analyses['13C NMR'].processed_data['peaks'].string_value = r'13.9, 27.6, 32.4, 41.0, 56.6, 59.1, 126.8, 128.2, 129.0, 140.2'

outcome.analyses['thin film IR'].type = analysis.IR
outcome.analyses['thin film IR'].details = 'neat film, NaCl'
outcome.analyses['thin film IR'].processed_data['peaks'].string_value = r'3063, 3027, 2964, 2870, 2791, 1602, 1494, 1453, 1365, 1244, 1121, 1069, 1028, 974, 745, 698 cm-1'

outcome.analyses['HRMS'].type = analysis.HRMS
outcome.analyses['HRMS'].details = 'ESI-TOF'
outcome.analyses['HRMS'].processed_data['expected'].float_value = 296.2373
outcome.analyses['HRMS'].processed_data['found'].float_value = 296.2375

outcome.analyses['quantitative NMR'].type = analysis.NMR_1H
outcome.analyses['quantitative NMR'].details = 'Quantitative NMR using ferrocene (98%, purchased from Sigma-Aldrich, recrystallized from pentane) in CDCl3'

outcome.analyses['chiral SFC'].type = analysis.SFC
outcome.analyses['chiral SFC'].details = 'FC (Daicel Chiralpak AD-H column, heated to 40 °C), eluting with a linear gradient over 6 min from 5% to 10% isopropanol in supercritical CO2 at 2.5 mL/min.: 2.57 min (major), 2.98 min (minor), 90% ee for both runs'

outcome.analyses['specific rotation'].type = analysis.CD
outcome.analyses['specific rotation'].details = 'Specific rotation: [α]D = -114 (c = 1.0, chloroform).'

product = outcome.products.add()
product.compound.CopyFrom(
    message_helpers.build_compound(
        smiles='C[C@@H](CN(CC1=CC=CC=C1)CC2=CC=CC=C2)C(C)(C)C', 
        name='(R)-N,N-Dibenzyl-2,3,3-trimethylbutan-1-amine', 
        amount='6.54 g'
    )
)
product.compound_yield.CopyFrom(
    reaction_pb2.Percentage(value=86)
)
product.selectivity.type = reaction_pb2.Selectivity.SelectivityType.EE
product.selectivity.value = 90
product.purity.value = 97

product.analysis_identity.extend(['1H NMR', '13C NMR', 'HRMS'])
product.analysis_yield.append('isolated_weight')
product.analysis_purity.append('quantitative NMR')
product.analysis_selectivity.extend(['chiral SFC', 'specific rotation'])

product.isolated_color = 'colorless'
product.texture.type = product.Texture.OIL

reaction.provenance.city = r'Cambridge, MA'
reaction.provenance.doi = r'10.15227/orgsyn.095.0080'
reaction.provenance.publication_url = r'http://www.orgsyn.org/demo.aspx?prep=v95p0080'
reaction.provenance.record_created.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
    name='Connor W. Coley', organization='MIT', orcid='0000-0002-8271-8723'
))

errors = validations.validate_message(reaction)
print(errors)
print(reaction)

[]
inputs {
  key: "(S)-DTBM-SEGPHOS"
  value {
    components {
      identifiers {
        type: SMILES
        value: "CC(C)(C)C1=CC(=CC(=C1OC)C(C)(C)C)P(C2=C(C3=C(C=C2)OCO3)C4=C(C=CC5=C4OCO5)P(C6=CC(=C(C(=C6)C(C)(C)C)OC)C(C)(C)C)C7=CC(=C(C(=C7)C(C)(C)C)OC)C(C)(C)C)C8=CC(=C(C(=C8)C(C)(C)C)OC)C(C)(C)C"
      }
      identifiers {
        type: NAME
        value: "(S)-DTBM-SEGPHOS"
      }
      mass {
        value: 330.0
        units: MILLIGRAM
      }
      reaction_role: REAGENT
      preparations {
        type: NONE
        details: "used as received (>94% purity)"
      }
      vendor_source: "Takasago"
    }
    addition_order: 3
  }
}
inputs {
  key: "2,3,3-trimethyl-1-butene"
  value {
    components {
      identifiers {
        type: SMILES
        value: "CC(=C)C(C)(C)C"
      }
      identifiers {
        type: NAME
        value: "2,3,3-trimethyl-1-butene"
      }
      mass {
        value: 2.5
        units: GRAM
      }
      reaction_role: REACTANT
      is_limiti